### Demo for local autoencoder creation & training
* steps on how table data can be slowly broken down and later fed to an autoencoder model
* steps later provide graphs on potential anomalies detected after recontruction (WIP)
* May use but BigQuery autoencoder seems more impactful/robust

In [ ]:
!pip install phonenumbers #https://pypi.org/project/phonenumbers/

In [ ]:
# Import libraries that will be needed for the lab
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import os, datetime
import glob
import pickle
import random

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, roc_curve, auc, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

import tensorflow as tf
from tensorflow import keras
from keras import optimizers, layers
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

In [ ]:
# Specify the path to your CSV file
csv_file_path = 'avoxi_workshop_bucket/large_table_cleaned/*.csv'
tmp_list = []

for csv_file in glob.glob(csv_file_path):
    # Read the CSV file into a DataFrame
    tmp_series = pd.read_csv(csv_file, delimiter=',', parse_dates=True)
    #filter some values
    # tmp_series
    # break
    tmp_list.append(tmp_series)

df = pd.concat(tmp_list)
#origin_df = df.copy(deep=True)

# Display the first few rows of the DataFrame
df

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df = df.drop(columns=['status'])

In [ ]:
df

In [ ]:
# Convert the `164_from_caller_id` and `164_to_caller_id` columns to 'Object' datatype
df['164_from_caller_id'] = df['164_from_caller_id'].astype(str)
df['164_to_caller_id'] = df['164_to_caller_id'].astype(str)

# Identify non-numeric columns
columns_to_exclude = ['day', 'hour'] # Most likely not needed for normalizing
non_numeric_cols = np.concatenate([df.select_dtypes(exclude=[np.number]).columns, columns_to_exclude])
numeric_cols = [col for col in df.select_dtypes(include=np.number) if col not in columns_to_exclude]

print(f"Numeric columns: {numeric_cols}")
print(f"Non-numeric columns: {non_numeric_cols}")

In [ ]:
# column names
# data types
# dataset memory usage
df.info()

In [ ]:
df.info()

In [ ]:
# Check for missing values in the dataframe and display the result
print("\nMissing Values:\n")
print(df.isnull().sum().to_markdown(numalign="left", stralign="left"))

In [ ]:
# Get the statistical summary of the numerical columns in the dataframe and display the result
print("\nStatistical Summary:\n")
print(df.describe().round(5).to_markdown(numalign="left", stralign="left"))

In [ ]:
import phonenumbers
from phonenumbers import geocoder

#https://github.com/azharkhn/libphonenumber-api/blob/master/phonenumber/lib/phonenumber.py
def get_E164format(phonenumber):
        #phonenumber = self.remove_chars_from_phonenumber(phonenumber)
        if(phonenumber[:2] == '00'):
            return '+1'+phonenumber
        else:
            return '+'+phonenumber  

def get_country_from_phone(phone_number, country="US"):
    #print(get_E164format(phone_number))
    formated_num = get_E164format(phone_number)
    try:
        country = geocoder.country_name_for_number(phonenumbers.parse(formated_num), "en")
        # Get the country name
        return country if country else "Invalid" #Empty is invalid
    except phonenumbers.phonenumberutil.NumberParseException:
        return "Invalid"

df['from_country'] = df['164_from_caller_id'].astype(str).apply(get_country_from_phone)
df['to_country'] = df['164_to_caller_id'].astype(str).apply(get_country_from_phone)
df.head()

In [ ]:
filtered_df = df.loc[(df['from_country'] != 'Invalid') & (df['to_country'] != 'Invalid')].copy(deep=True)
filtered_df

In [ ]:
filtered_df = filtered_df.drop(columns=['status'])

In [ ]:
filtered_df

In [ ]:
filtered_df[numeric_cols].describe()

In [ ]:
anomalies_df = filtered_df.loc[((filtered_df['packet_loss'] >= 0.1) 
                                & (filtered_df['jitter'] > filtered_df['jitter'].quantile(0.50))) 
                               & (filtered_df['mean_opinion_score'] < filtered_df['mean_opinion_score'].quantile(0.25))].copy(deep=True)
anomalies_df['is_anomaly'] = True
anomalies_df

In [ ]:
no_anomalies_df = filtered_df.loc[(filtered_df['duration'] > filtered_df['duration'].quantile(0.5)) 
                                                & (filtered_df['mean_opinion_score'] > 4) 
                                                & (filtered_df['jitter'] <= 1) 
                                                & (filtered_df['packet_loss'] <= 0)].copy()
no_anomalies_df['is_anomaly'] = False
no_anomalies_df

In [ ]:
contamination = (anomalies_df.shape[0]/(anomalies_df.shape[0] + no_anomalies_df.shape[0]))
print(f"Contamination: {int(contamination*100)}%") 

In [ ]:
train_test_df = pd.concat([anomalies_df, no_anomalies_df])
train_test_df

In [ ]:
def normalize_dataset(df):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df[numeric_cols])
    scaled_df = pd.DataFrame(scaled_data, columns=numeric_cols, index=df.index)
    result_df = pd.concat([scaled_df, df[non_numeric_cols]], axis=1)
    print("Processed DataFrame; normalized Dataframe created")
    return result_df

In [ ]:
train_test_df = normalize_dataset(train_test_df)
train_test_df

In [ ]:
train_test_df.info()

In [ ]:
train_test_df.isnull().sum()

In [ ]:
# Split data into train and test sets
#X_train, X_test = train_test_split(train_test_df[numeric_cols], test_size=0.1, random_state=42)
X_train = train_test_df[numeric_cols]

# Input layer
input_shape = X_train.shape[1]
input_layer = Input(shape=(input_shape,))

In [ ]:
X_train

In [ ]:
X_test.info()

In [ ]:
input_layer

In [ ]:
# Encoding layer (hidden layer)
# Input layer
input_shape = X_train.shape[1]
input_layer = Input(shape=(input_shape,))
encoded = Dense(input_shape, activation='relu')(input_layer)
encoded = Dense(int(input_shape/2), activation='relu')(encoded)

# Decoding layer (output layer)
decoded = Dense(int(input_shape/2), activation='relu')(encoded)
decoded = Dense(input_shape, activation='sigmoid')(decoded)

# Autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mse')

# Train the model
nb_epoch = 100
batch_size = 32
#history = autoencoder.fit(X_train, X_train, epochs=nb_epoch, batch_size=batch_size, validation_data=(X_test, X_test))
history = autoencoder.fit(X_train, X_train, epochs=nb_epoch, batch_size=batch_size)

# Print the model summary
autoencoder.summary()

In [ ]:
plt.plot(history.history['loss'])
plt.legend(['loss on train data', 'loss on validation data'])

In [ ]:
dataset_size = 1000
X_origin = filtered_df.sample(n=dataset_size).copy(deep=True)
X_test = normalize_dataset(X_origin)
X_test

In [ ]:
x_test_recon = autoencoder.predict(X_test[numeric_cols])

# the reconstruction score is the mean of the reconstruction errors (relatively high scores are anomalous)
reconstruction_scores = np.mean((X_test[numeric_cols] - x_test_recon)**2, axis=1)

In [ ]:
anomaly_data = pd.DataFrame({'recon_score':reconstruction_scores})

# if our reconstruction scores our normally distributed we can use their statistics
anomaly_data.info()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import erf

# calculate mean and standard deviation
mse = np.mean(np.square(X_test[numeric_cols] - x_test_recon), axis=1)
threshold = np.mean(mse) + 1 * np.std(mse)
print(f"Threshold: {threshold}")
anomalies = np.where(mse > threshold)[0]

print(anomalies)
#print(mse.iloc[265])

mean_mse = np.mean(mse)
std_mse = np.std(mse)
# define thresholds based on number of standard deviations away from the mean
threshold_2std = mean_mse + 2 * std_mse
threshold_3std = mean_mse + 3 * std_mse
threshold_4std = mean_mse + 4 * std_mse
threshold_5std = mean_mse + 5 * std_mse
confidences = [100*erf(i/np.sqrt(2)) for i in range(2, 6)]
# create a scatter plot of the reconstruction error vs sample index
plt.scatter(range(len(mse)), mse)
# highlight the anomalies
plt.scatter(anomalies, mse.iloc[anomalies], color='red') #anomaly within anomalies (intended behavior?)
# add threshold lines
plt.axhline(y=threshold_2std, color='green', linestyle='--', label='2 std')
plt.text(0.02, threshold_2std + 0.2, f"2σ ({confidences[0]:.2f}% confidence)")
plt.axhline(y=threshold_3std, color='orange', linestyle='--', label='3 std')
plt.text(0.02, threshold_3std + 0.2, f"3σ ({confidences[1]:.2f}% confidence)")
plt.axhline(y=threshold_4std, color='blue', linestyle='--', label='4 std')
plt.text(0.02, threshold_4std + 0.2, f"4σ ({confidences[2]:.2f}% confidence)")
plt.axhline(y=threshold_5std, color='purple', linestyle='--', label='5 std')
plt.text(0.02, threshold_5std + 0.2, f"5σ ({confidences[3]:.2f}% confidence)")
# add labels and title
plt.xlabel('Record Index')
plt.ylabel('Reconstruction Error (MSE)')
plt.title('Anomaly Detection Results')
# display the plot
plt.show()

In [ ]:
X_origin.loc[~X_origin.isin(X_origin.iloc[anomalies]).all(axis=1)] #Not anomalies

In [ ]:
X_origin.loc[X_origin.isin(X_origin.iloc[anomalies]).all(axis=1)] #anomalies

In [ ]:
X_origin['is_anomaly'] = False
X_origin['is_anomaly'].iloc[anomalies] = True
X_origin

In [ ]:
autoencoder.save('avoxi_workshop_bucket/saved_model/autoencoder_v3')

In [ ]:
from google.cloud import bigquery

def insert_dataframe_as_table(project_id, dataset_id, table_id, dataframe):
    """
    Inserts a pandas DataFrame into a BigQuery table.

    Args:
        project_id (str): The Google Cloud project ID.
        dataset_id (str): The BigQuery dataset ID.
        table_id (str): The BigQuery table ID.
        dataframe (pandas.DataFrame): The DataFrame to insert.
    """

    client = bigquery.Client(project=project_id)

    # Determine schema from DataFrame if not explicitly provided
    job_config = bigquery.LoadJobConfig(
        autodetect=True, # Autodetect schema from DataFrame
        source_format=bigquery.SourceFormat.PARQUET,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE # Overwrite existing table if it exists. 
    )

    # Load data from DataFrame
    job = client.load_table_from_dataframe(
        dataframe, f"{project_id}.{dataset_id}.{table_id}", job_config=job_config
    )

    job.result()  # Wait for the job to complete
    print(f"Loaded {job.output_rows} rows into {dataset_id}:{table_id}")

# Example Usage (replace with your actual data)
project_id = "gurkomal-playground"
dataset_id = "avoxi_workshop"
table_id = "anomaly_labelled_data_v2"
dataframe = ...  # Your Pandas DataFrame

insert_dataframe_as_table(project_id, dataset_id, table_id, train_test_df)